In [ ]:
pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO

In [ ]:
# Load YOLOv8 model (n = nano version for better speed on low-end devices)
model = YOLO("yolov8n.pt")

In [ ]:
# Classes from COCO dataset that we want to detect and count
COCO_CLASSES = {
    0: "person",
    2: "car",
    3: "motorcycle",
    16: "dog",
    39: "bottle",
    56: "chair"
}

In [ ]:
# Counter for each class
object_counts = {cls_name: 0 for cls_name in COCO_CLASSES.values()}

In [ ]:
# Store previous center points to avoid double counting
past_centers = {cls_id: [] for cls_id in COCO_CLASSES}

In [ ]:
# Function to get center of bounding box
def get_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

In [ ]:
# Start webcam
cap = cv2.VideoCapture(0)  # 0 for default webcam

In [ ]:
# Define line position for counting
line_position = 250

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 detection
    results = model(frame, stream=True)

    # Draw the horizontal line on screen
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (255, 0, 0), 2)

    # Process results
    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])

            # Only detect if it's a class we're interested in and confidence is high
            if cls_id in COCO_CLASSES and conf > 0.5:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cx, cy = get_center(x1, y1, x2, y2)
                cls_name = COCO_CLASSES[cls_id]

                # Draw box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{cls_name} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Check if object crosses the line and hasn't been counted before
                if line_position - 10 < cy < line_position + 10:
                    already_counted = any(abs(cx - pcx) < 20 and abs(cy - pcy) < 20
                                          for pcx, pcy in past_centers[cls_id])
                    if not already_counted:
                        object_counts[cls_name] += 1
                        past_centers[cls_id].append((cx, cy))

    # Show object counts on screen
    y_offset = 30
    for obj, cnt in object_counts.items():
        cv2.putText(frame, f"{obj.capitalize()}: {cnt}", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        y_offset += 25

    # Show the output frame
    cv2.imshow("Object Detection and Counting", frame)

    # Exit with ESC key
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
